In [1]:
import pandas as pd
import numpy as np
from model.gmf import GMFEngine
from model.mlp import MLPEngine
from model.neumf import NeuMFEngine
from data import SampleGenerator
import os

In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
# DataLoader for training
thepath=os.path.join('data','user-item-oct-dec-2018-v2-modified.csv')
sample_generator = SampleGenerator(thepath)


Initialising data cleaning
Number of bad users: 75054 
Number of bad items: 72173
Starting clean process 1

Number of bad users: 5553 
Number of bad items: 7677
Starting clean process 2

Number of bad users: 1542 
Number of bad items: 1906
Starting clean process 3

Number of bad users: 465 
Number of bad items: 672
Starting clean process 4

Number of bad users: 199 
Number of bad items: 243
Starting clean process 5

Number of bad users: 66 
Number of bad items: 104
Starting clean process 6

Number of bad users: 32 
Number of bad items: 40
Starting clean process 7

Number of bad users: 9 
Number of bad items: 19
Starting clean process 8

Number of bad users: 5 
Number of bad items: 4
Starting clean process 9

Number of bad users: 3 
Number of bad items: 4
Starting clean process 10

Number of bad users: 1 
Number of bad items: 3
Starting clean process 11

Number of bad users: 1 
Number of bad items: 1
Starting clean process 12

Number of bad users: 1 
Number of bad items: 1
Starting cle

  3%|▎         | 330/11410 [00:00<00:03, 3292.80it/s]

1


  2%|▏         | 254/11410 [00:00<00:05, 2143.57it/s]

2


100%|██████████| 11410/11410 [00:03<00:00, 3585.30it/s]

split_loo
train user n unique: 11410
test user n unique: 11410
Done!


In [4]:
evaluate_data = sample_generator.evaluate_data

Begin the loop...


In [6]:
num_itemid, num_userid = sample_generator.usr_item_unique()

In [7]:
num_itemid

13203

In [8]:
num_userid

11410

In [9]:
# Training Engine
def train_model(model, config):
    engine = model(config)
    best_hit = 0
    for epoch in range(config['num_epoch']):
        print('Epoch {} starts !'.format(epoch))
        print('-' * 70)
        train_loader = sample_generator.instance_a_train_loader(config['num_negative'], config['batch_size'])
        engine.train_an_epoch(train_loader, epoch_id=epoch)
        hit_ratio, ndcg = engine.evaluate(evaluate_data, epoch_id=epoch)
        if epoch % 20 == 0:
            engine.save(config['alias'], epoch, hit_ratio, ndcg)
        elif (epoch == config['num_epoch'] - 1):
            engine.save(config['alias'], epoch, hit_ratio, ndcg)
        if hit_ratio > best_hit:
            best_hit = hit_ratio
            engine.save(config['alias'], epoch, hit_ratio, ndcg, backup=False)

In [ ]:
#setup configuration for GMF
gmf_config = {'alias': 'gmf_factor8neg4-implict',
              'num_epoch': 200,
              'batch_size': 4,
              # 'optimizer': 'sgd',
              # 'sgd_lr': 1e-3,
              # 'sgd_momentum': 0.9,
              # 'optimizer': 'rmsprop',
              # 'rmsprop_lr': 1e-3,
              # 'rmsprop_alpha': 0.99,
              # 'rmsprop_momentum': 0,
              'optimizer': 'adam',
              'adam_lr': 1e-3,
              'num_users': num_userid,
              'num_items': num_itemid,
              'latent_dim': 8,
              'num_negative': 4,
              'l2_regularization': 0, # 0.01
              'use_cuda': True,
              'device_id': 0,
              'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'}

In [ ]:
# Train GMF Model
train_model(GMFEngine, gmf_config)

Epoch 0 starts !
----------------------------------------------------------------------
[Training Stage Epoch 0] Loss 0.5944649577140808


/home/ec2-user/SageMaker/tdc-product-recommendation/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


[Evaluating Epoch 0] HR = 0.0940, NDCG = 0.0430

Epoch 1 starts !
----------------------------------------------------------------------
[Training Stage Epoch 1] Loss 0.5706994533538818
[Evaluating Epoch 1] HR = 0.1037, NDCG = 0.0477

Epoch 2 starts !
----------------------------------------------------------------------
[Training Stage Epoch 2] Loss 0.22941476106643677
[Evaluating Epoch 2] HR = 0.1019, NDCG = 0.0457

Epoch 3 starts !
----------------------------------------------------------------------
[Training Stage Epoch 3] Loss 0.5677672028541565
[Evaluating Epoch 3] HR = 0.1054, NDCG = 0.0478

Epoch 4 starts !
----------------------------------------------------------------------
[Training Stage Epoch 4] Loss 0.5617380738258362
[Evaluating Epoch 4] HR = 0.1079, NDCG = 0.0499

Epoch 5 starts !
----------------------------------------------------------------------
[Training Stage Epoch 5] Loss 0.5944874882698059
[Evaluating Epoch 5] HR = 0.1098, NDCG = 0.0520

Epoch 6 starts !
---

In [10]:
mlp_config = {'alias': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001',
              'num_epoch': 200,
              'batch_size': 4,  # 1024,
              'optimizer': 'adam',
              'adam_lr': 1e-3,
              'num_users': num_userid,
              'num_items': num_itemid,
              'latent_dim': 8,
              'num_negative': 4,
              'layers': [16,64,32,16,8],  # layers[0] is the concat of latent user vector & latent item vector
              'l2_regularization': 0.0000001,  # MLP model is sensitive to hyper params
              'use_cuda': True,
              'device_id': 0,
              'pretrain': True,
              'pretrain_mf': 'gmf_factor8neg4-implict_best.model',
              'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'}

In [11]:
train_model(MLPEngine, mlp_config)
#need to add a learning rate scheduler

MLP(
  (embedding_user): Embedding(11411, 8)
  (embedding_item): Embedding(13203, 8)
  (fc_layers): ModuleList(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Linear(in_features=16, out_features=8, bias=True)
  )
  (affine_output): Linear(in_features=8, out_features=1, bias=True)
  (logistic): Sigmoid()
)
Epoch 0 starts !
----------------------------------------------------------------------
[Training Stage Epoch 0] Loss 0.3539768159389496


/home/ec2-user/SageMaker/tdc-product-recommendation/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


[Evaluating Epoch 0] HR = 0.2777, NDCG = 0.1935

Epoch 1 starts !
----------------------------------------------------------------------
[Training Stage Epoch 1] Loss 0.7906252145767212
[Evaluating Epoch 1] HR = 0.2894, NDCG = 0.2020

Epoch 2 starts !
----------------------------------------------------------------------
[Training Stage Epoch 2] Loss 0.9205023050308228
[Evaluating Epoch 2] HR = 0.2815, NDCG = 0.1973

Epoch 3 starts !
----------------------------------------------------------------------
[Training Stage Epoch 3] Loss 0.2467784285545349
[Evaluating Epoch 3] HR = 0.2856, NDCG = 0.1999

Epoch 4 starts !
----------------------------------------------------------------------
[Training Stage Epoch 4] Loss 0.21626172959804535
[Evaluating Epoch 4] HR = 0.2894, NDCG = 0.1993

Epoch 5 starts !
----------------------------------------------------------------------
[Training Stage Epoch 5] Loss 0.7835968732833862
[Evaluating Epoch 5] HR = 0.2901, NDCG = 0.2022

Epoch 6 starts !
---

[Training Stage Epoch 44] Loss 0.44563227891921997
[Evaluating Epoch 44] HR = 0.2905, NDCG = 0.2004

Epoch 45 starts !
----------------------------------------------------------------------
[Training Stage Epoch 45] Loss 0.41291603446006775
[Evaluating Epoch 45] HR = 0.2949, NDCG = 0.2008

Epoch 46 starts !
----------------------------------------------------------------------
[Training Stage Epoch 46] Loss 0.6591451168060303
[Evaluating Epoch 46] HR = 0.2901, NDCG = 0.1965

Epoch 47 starts !
----------------------------------------------------------------------
[Training Stage Epoch 47] Loss 0.43042346835136414
[Evaluating Epoch 47] HR = 0.2976, NDCG = 0.2048

Epoch 48 starts !
----------------------------------------------------------------------
[Training Stage Epoch 48] Loss 0.26253604888916016
[Evaluating Epoch 48] HR = 0.3046, NDCG = 0.2066

Epoch 49 starts !
----------------------------------------------------------------------
[Training Stage Epoch 49] Loss 0.6797062158584595
[

[Evaluating Epoch 87] HR = 0.2853, NDCG = 0.1933

Epoch 88 starts !
----------------------------------------------------------------------
[Training Stage Epoch 88] Loss 0.34051835536956787
[Evaluating Epoch 88] HR = 0.2912, NDCG = 0.2026

Epoch 89 starts !
----------------------------------------------------------------------
[Training Stage Epoch 89] Loss 0.7296736836433411
[Evaluating Epoch 89] HR = 0.2910, NDCG = 0.2022

Epoch 90 starts !
----------------------------------------------------------------------
[Training Stage Epoch 90] Loss 0.7484243512153625
[Evaluating Epoch 90] HR = 0.2898, NDCG = 0.1997

Epoch 91 starts !
----------------------------------------------------------------------
[Training Stage Epoch 91] Loss 0.11499305814504623
[Evaluating Epoch 91] HR = 0.2946, NDCG = 0.2043

Epoch 92 starts !
----------------------------------------------------------------------
[Training Stage Epoch 92] Loss 0.036611296236515045
[Evaluating Epoch 92] HR = 0.2914, NDCG = 0.2008

E

[Evaluating Epoch 130] HR = 0.2828, NDCG = 0.1987

Epoch 131 starts !
----------------------------------------------------------------------
[Training Stage Epoch 131] Loss 0.21749499440193176
[Evaluating Epoch 131] HR = 0.2852, NDCG = 0.2013

Epoch 132 starts !
----------------------------------------------------------------------
[Training Stage Epoch 132] Loss 0.10274486988782883
[Evaluating Epoch 132] HR = 0.2860, NDCG = 0.2009

Epoch 133 starts !
----------------------------------------------------------------------
[Training Stage Epoch 133] Loss 0.049858611077070236
[Evaluating Epoch 133] HR = 0.2912, NDCG = 0.2025

Epoch 134 starts !
----------------------------------------------------------------------
[Training Stage Epoch 134] Loss 0.5141432285308838
[Evaluating Epoch 134] HR = 0.2743, NDCG = 0.1981

Epoch 135 starts !
----------------------------------------------------------------------
[Training Stage Epoch 135] Loss 0.4199206233024597
[Evaluating Epoch 135] HR = 0.2804, 

[Training Stage Epoch 173] Loss 0.14785030484199524
[Evaluating Epoch 173] HR = 0.2973, NDCG = 0.1984

Epoch 174 starts !
----------------------------------------------------------------------
[Training Stage Epoch 174] Loss 0.07103881984949112
[Evaluating Epoch 174] HR = 0.2954, NDCG = 0.1794

Epoch 175 starts !
----------------------------------------------------------------------
[Training Stage Epoch 175] Loss 0.06209569424390793
[Evaluating Epoch 175] HR = 0.2950, NDCG = 0.1927

Epoch 176 starts !
----------------------------------------------------------------------
[Training Stage Epoch 176] Loss 0.1618412882089615
[Evaluating Epoch 176] HR = 0.2953, NDCG = 0.1941

Epoch 177 starts !
----------------------------------------------------------------------
[Training Stage Epoch 177] Loss 0.15147936344146729
[Evaluating Epoch 177] HR = 0.3013, NDCG = 0.2001

Epoch 178 starts !
----------------------------------------------------------------------
[Training Stage Epoch 178] Loss 0.57

In [12]:
neumf_config = {'alias': 'pretrain_neumf_factor8neg4',
                'num_epoch': 200,
                'batch_size': 4,
                'optimizer': 'adam',
                'adam_lr': 1e-3,
                'num_users': num_userid,
              'num_items': num_itemid,
                'latent_dim_mf': 8,
                'latent_dim_mlp': 8,
                'num_negative': 4,
                'layers': [16,64,32,16,8],  # layers[0] is the concat of latent user vector & latent item vector
                'l2_regularization': 0.0000001,
                'use_cuda': True,
                'device_id': 0,
                'pretrain': True,
                'pretrain_mf': 'gmf_factor8neg4-implict_best.model',
                'pretrain_mlp': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001_best.model',
                'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'
                }

In [ ]:
train_model(NeuMFEngine, neumf_config)

NeuMF(
  (embedding_user_mlp): Embedding(11411, 8)
  (embedding_item_mlp): Embedding(13203, 8)
  (embedding_user_mf): Embedding(11411, 8)
  (embedding_item_mf): Embedding(13203, 8)
  (fc_layers): ModuleList(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Linear(in_features=16, out_features=8, bias=True)
  )
  (affine_output): Linear(in_features=16, out_features=1, bias=True)
  (logistic): Sigmoid()
)
Epoch 0 starts !
----------------------------------------------------------------------
[Training Stage Epoch 0] Loss 0.2156829684972763


/home/ec2-user/SageMaker/tdc-product-recommendation/metrics.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1


[Evaluating Epoch 0] HR = 0.2520, NDCG = 0.1753

Epoch 1 starts !
----------------------------------------------------------------------
[Training Stage Epoch 1] Loss 0.16223248839378357
[Evaluating Epoch 1] HR = 0.2661, NDCG = 0.1891

Epoch 2 starts !
----------------------------------------------------------------------
[Training Stage Epoch 2] Loss 0.6058328151702881
[Evaluating Epoch 2] HR = 0.2681, NDCG = 0.1910

Epoch 3 starts !
----------------------------------------------------------------------
[Training Stage Epoch 3] Loss 0.21760070323944092
[Evaluating Epoch 3] HR = 0.2789, NDCG = 0.1966

Epoch 4 starts !
----------------------------------------------------------------------
[Training Stage Epoch 4] Loss 0.9215611219406128
[Evaluating Epoch 4] HR = 0.2788, NDCG = 0.1975

Epoch 5 starts !
----------------------------------------------------------------------
[Training Stage Epoch 5] Loss 0.3125734329223633
[Evaluating Epoch 5] HR = 0.2788, NDCG = 0.1949

Epoch 6 starts !
--

[Training Stage Epoch 44] Loss 0.46183568239212036
[Evaluating Epoch 44] HR = 0.2928, NDCG = 0.2012

Epoch 45 starts !
----------------------------------------------------------------------
[Training Stage Epoch 45] Loss 0.6940931081771851
[Evaluating Epoch 45] HR = 0.2937, NDCG = 0.2028

Epoch 46 starts !
----------------------------------------------------------------------
[Training Stage Epoch 46] Loss 0.20783080160617828
[Evaluating Epoch 46] HR = 0.2884, NDCG = 0.2014

Epoch 47 starts !
----------------------------------------------------------------------
[Training Stage Epoch 47] Loss 0.061200037598609924
[Evaluating Epoch 47] HR = 0.2900, NDCG = 0.2000

Epoch 48 starts !
----------------------------------------------------------------------
[Training Stage Epoch 48] Loss 0.11163555085659027
[Evaluating Epoch 48] HR = 0.2885, NDCG = 0.1972

Epoch 49 starts !
----------------------------------------------------------------------
[Training Stage Epoch 49] Loss 0.8130155801773071


In [ ]:
print('Done!')

In [13]:
neumf_config_d = {'alias': 'pretrain_neumf_factor8neg4',
                'num_epoch': 200,
                'batch_size': 4,
                'optimizer': 'adam',
                'adam_lr': 1e-3,
                'num_users': num_userid,
              'num_items': num_itemid,
                'latent_dim_mf': 8,
                'latent_dim_mlp': 8,
                'num_negative': 4,
                'layers': [16,64,32,16,8],  # layers[0] is the concat of latent user vector & latent item vector
                'l2_regularization': 0.0000001,
                'use_cuda': False,
                'device_id': 0,
                'pretrain': True,
                'pretrain_mf': 'gmf_factor8neg4-implict_best.model',
                'pretrain_mlp': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001_best.model',
                'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'
                }

In [14]:
model=NeuMFEngine(neumf_config_d)

NeuMF(
  (embedding_user_mlp): Embedding(11411, 8)
  (embedding_item_mlp): Embedding(13203, 8)
  (embedding_user_mf): Embedding(11411, 8)
  (embedding_item_mf): Embedding(13203, 8)
  (fc_layers): ModuleList(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): Linear(in_features=16, out_features=8, bias=True)
  )
  (affine_output): Linear(in_features=16, out_features=1, bias=True)
  (logistic): Sigmoid()
)


RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at /opt/conda/conda-bld/pytorch_1549281116128/work/torch/csrc/cuda/Module.cpp:53